In [147]:
import pandas as pd
train_user = pd.read_csv("data/trainingset/train_base.csv")
train_op = pd.read_csv("data/trainingset/train_op.csv")
train_trans = pd.read_csv("data/trainingset/train_trans.csv")
train_label = pd.read_csv("data/trainingset/train_label.csv")


In [148]:
test_user = pd.read_csv("data/testingset/test_a_base.csv")
test_op = pd.read_csv("data/testingset/test_a_op.csv")
test_trans = pd.read_csv("data/testingset/test_a_trans.csv")

In [149]:
def factorize(data, cols):
    '''
    对类别变量进行onehot编码
    '''
    for col in cols:
        dummy_col = pd.get_dummies(data[col], dummy_na=True, prefix=col + "_")
        data = pd.merge(data,dummy_col,left_index=True, right_index=True)
    data.drop(cols,axis=1,inplace=True)
    return data

def split_str_to_int(data, cols):
    for col in cols:
        data[col].fillna("catetory -1", inplace=True)
        data[col] = data[col].map(lambda x: int(x.split(" ")[1]))
        
def minus_min(data, cols):
    for col in cols:
        min_value = data[col].min()
        data[col] = data[col].map(lambda x: x - min_value)    

In [150]:
def transform_feature(data):
    copy_data = data.copy(deep=True)
    copy_data = factorize(copy_data, factorize_cols)
    split_str_to_int(copy_data, split_str_to_int_cols)
    minus_min(copy_data,minus_min_cols)
    return copy_data

In [151]:
from sklearn.model_selection import KFold

def kfold_predict(train_X, train_Y, test_X, max_iter=100):
    scores = []
    kf = KFold(n_splits=5)
    kf.get_n_splits(train_X)
    predict_Y = []
    for train_index, test_index in kf.split(train_X):
        X_train, X_test = train_X.loc[train_index], train_X.loc[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        model = LogisticRegression(max_iter=max_iter)
        model.fit(X_train, y_train)
        outcome = model.predict_proba(X_test)
        score = roc_auc_score(y_test, outcome[:,1])
        scores.append(score)
        predict_Y.append(list(model.predict_proba(test_X)[:,1]))
    y_pred = np.mean(np.array(predict_Y), axis=0)
        
    return np.mean(scores), y_pred
        

In [154]:
#未加入city in factorize_cols
factorize_cols = ["sex", "provider", "verified", "regist_type", "agreement1", "agreement2", "agreement3", \
                  "agreement4", "province","service3"]
split_str_to_int_cols = ["balance", "level", "balance_avg", "balance2", "balance2_avg", \
                        "service3_level", "product1_amount", "product2_amount", "product3_amount", \
                        "product4_amount", "product5_amount", "product6_amount", "balance1_avg", \
                        "balance1"]
minus_min_cols = ["age","card_a_cnt","card_b_cnt","card_c_cnt","card_d_cnt","op1_cnt","op2_cnt","agreement_total",\
                 "service1_cnt","service1_amt","service2_cnt","acc_count","login_cnt_period1","login_cnt_period2",\
                 "ip_cnt","login_cnt_avg","login_days_cnt","product7_cnt","product7_fail_cnt","using_time"]

In [155]:
transformed_train_user = transform_feature(train_user)
transformed_test_user = transform_feature(test_user)
# transformed_total_user_data = transform_feature(total_user_data)
# transformed_train_data = transformed_total_user_data.iloc[:train_data_len,:]
# transformed_test_data = transformed_total_user_data.iloc[train_data_len:,:]

In [156]:
# 模型训练
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

final_input_data = pd.merge(transformed_train_user, train_label, on="user", how="right")

input_cols = list(transformed_train_user.columns)
input_cols.remove("user")
input_cols.remove("city")
# input_cols.remove("label")

train_Y = final_input_data["label"]
train_X = final_input_data[input_cols]

test_X = transformed_test_user[input_cols]
y_offline, y_pred = kfold_predict(train_X, train_Y, test_X, max_iter=100)
print(y_offline)

/Users/feedliu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/feedliu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/feedliu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/feedliu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/feedliu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed

0.6785898743196612


In [157]:
# 生成提交文件
test_user["label"] = pd.Series(y_pred)

test_user[["user", "label"]].to_csv("baseline4.csv", header=["user", "prob"], index=False)